In [2]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [11]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [4]:
text = f"""
The Anatomic Guide® is a groundbreaking bone reduction guide for full-arch guided implant placement surgery. \ 
It’s a completely-guided, stackable system that gives you unmatched accuracy and predictability in terms of restorative treatment outcomes.
"""

prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence less than 10 words.
```{text}```
"""
response = get_completion(prompt)
print(response)

The Anatomic Guide® is a revolutionary bone reduction guide.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [13]:
prompt = f"""
Generate a list of three made-up users along \ 
with their email, password and profile images links. 
Provide them in JSON format with the following keys: 
firstname, lastname, username, email, password, profileImageURL.
"""
response = get_completion(prompt)
print(response)

{
  "users": [
    {
      "firstname": "John",
      "lastname": "Doe",
      "username": "johndoe",
      "email": "johndoe@example.com",
      "password": "password123",
      "profileImageURL": "https://example.com/profiles/johndoe.jpg"
    },
    {
      "firstname": "Jane",
      "lastname": "Smith",
      "username": "janesmith",
      "email": "janesmith@example.com",
      "password": "pass123",
      "profileImageURL": "https://example.com/profiles/janesmith.jpg"
    },
    {
      "firstname": "Mike",
      "lastname": "Johnson",
      "username": "mikejohnson",
      "email": "mikejohnson@example.com",
      "password": "mikepass",
      "profileImageURL": "https://example.com/profiles/mikejohnson.jpg"
    }
  ]
}


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [18]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [21]:
prompt = f"""
Tell me about Smart Roz Blbn Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The Smart Roz Blbn Toothbrush by Boie is a revolutionary toothbrush that combines advanced technology with sustainable design. It is designed to provide a superior brushing experience while minimizing environmental impact.

One of the standout features of the Smart Roz Blbn Toothbrush is its sonic vibration technology. It utilizes high-frequency vibrations to effectively clean teeth and gums, removing plaque and promoting oral health. The brush head is designed with soft, rounded bristles that are gentle on the gums and enamel, ensuring a comfortable brushing experience.

The Smart Roz Blbn Toothbrush also incorporates smart features that enhance the brushing process. It has a built-in timer that ensures you brush for the recommended two minutes, evenly dividing the time between four quadrants of your mouth. This helps you maintain a consistent brushing routine and ensures thorough cleaning.

In addition to its advanced technology, the Smart Roz Blbn Toothbrush is also eco-friendly. It

In [22]:
prompt = f"""
Tell me about Smart Roz Blbn Toothbrush by Boie, firstly validate if this is a real product or not, if not, tell me that that is not a real product.
"""
response = get_completion(prompt)
print(response)

I'm sorry, but I couldn't find any information about a product called "Smart Roz Blbn Toothbrush by Boie." It is possible that this product does not exist.


## ---------------------------------------------

## Iterative Prompting ####

In [24]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Egypt
"""

In [28]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website to be readable for human. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <h2>Product Description</h2>
  <p>
    Introducing our latest addition to our mid-century inspired office furniture collection, the SWC-100 Chair. This chair is part of a beautiful family of furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. With its sleek design and customizable options, it is perfect for both home and business settings.
  </p>
  <p>
    The SWC-100 Chair is available in several options of shell color and base finishes, allowing you to choose the perfect combination to match your space. You can opt for plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. Additionally, you have the choice of having armrests or going armless.
  </p>
  <p>
    Constructed with durability and comfort in mind, the SWC-100 Chair features a 5-wheel plastic coated aluminum base for stability and mobility. The chair also has

In [29]:
from IPython.display import display, HTML
display(HTML(response))

Dimension,Measurement (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""


## ---------------------------------------------

## Summarizing

In [30]:
prod_review = """
After finally purchasing a couple MiniPC's for both business and personal use, and using an old LCD 21" Wide Screen Monitor I had, I was finding that I must be spoiled by my 4K TV because the picture on that monitor which I thought was wonderful, just seemed so dull and it did not support 4K which I have come to love to watch 4K videos etc.
I wasn't really sure what monitor to get, but I wanted something much larger, supported 4K, and that was at a very reasonable price. I did a lot of searching Amazon's product lists for monitors, and I came across this monitor, and I noticed that it was a curved screen. While I have never owned a curved screen anything, I had seen them at the local electronics store and was darn impressed. So I was very curious about this monitor because it had all the specs I was looking for, at least 27", Wide Screen, Support for 4K, Native Resolution of 1920 x 1080, LED, and had a curved screen. When I also noticed that it was a Samsung, I was pretty excited to learn more because I have always had very good luck with Samsung products and in fact, I have only had one Samsung product ever fail, and that was a mobile phone that should have stopped working years ago with what It went through.
Not knowing much about monitors these days, I started reading the reviews that were written on this monitor and I asked the community a couple questions. The answers I got back to my questions sealed the deal for me. Everyone who responded had nothing but great things to say about this monitor and provided me in info I needed. So I decided to go for it.
When it got here, unfortunately the first one had a broken screen. It appeared that it was packaged that way since the packaging was factory sealed but the packing was broken on the inside. Needless to say I returned it but Amazon really stepped up to the plate and sent me a replacement next day, so I was only delayed by 1 day getting to use it. Because the first monitor came with a broken screen, I am going to assume that the screen may be a bit on the frgile side as my old monitor has been dropped, and the LCD screen never had any issues, I don't know jf the screens on LED monitors are more easily broken, or If this was something that someone really dropped hard before packaging, for this reason I gave the durability 4 stars)
Setup was a breeze! Just had to put 1 screw to put the base stand together, and that was it. The stand is about the only thing I really don't like because the base that sits on the desk is an upside down chevron shape, which sticks out in front of the monitor a few inches. This makes putting it on a small desk where your keyboard would be in front of it a bit of a pain because the keyboard has to be pulled closer to me and just gets in the way to me. I managed to get around that by putting the monitor up on a little shelf I made out of a box and an old Laptop lap desk that I had. It perfectly raises the monitor by about 4-5 inches, and let's me push the keyboard further back to where I now have more room on my desk. (Images added to show the base and how I worked around the chevron styled base to give me more room).
The monitor has one HDMI port and one Display Port. Now I am not familiar with the Display Port, but all my devices are HDMI, so plugging in a PC is simple. In fact, I have 3 MiniPC's that I need to connect to this monitor so I bought a 5 port HDMI switch. This allows me to easily switch between each MiniPC with just a click of a button.
The picture on this monitor is absolutely phenomenal! Once I plugged in the HDMI plug, turned on the monitor, it automatically scanned for the signal which it found right away and WOW! What a gorgeous picture. The colors are very bright and vibrant. The dynamics between the Black and White colors are stunning and the color definition is very vivid. For never using a curved screen before on a computer monitor, I am very impressed. It almost immerses you in to the picture and almost gives it a extra dimensional view, almost 3D in nature... The thing I worried most about with a curved screen was is there going to be any distortion with a curved screen and I am happy to say. "Absolutely Not!". The images looks perfectly flat, but because of the curve, it pulls the outer edges our of your peripheral vision and closer to your focused view when looking straight forward.
Learning how to use the toggle button on the back for adjusting the settings van take a bit of getting used to, however, at this point, I haven't really needed to use any of the options other then Sleep and Shut Down as the defaults seem to be perfect for my setup.
Unfortunately this monitor does not have built in speakers however, if you have speakers connected to you computer or use a set of Bluetooth Speakers you will be just fine.
Overall my impression of this monitor is fantastic! The picture quality is stunning, and the size with the curves screen is awesome. I would recommend this monitor to anyone who wants an excellent monitor for home or office at a more then reasonable price. While I have not used the monitor for gaming, I have used a monitor that has the same 4ms refresh rate and that has the same native resolution of1920 x 1080 and that monitor is over 10 years old and still holds up today, so I can only imagine that this monitor would be perfectly fine for today's games. I am definitely glad that I went ahead and purchased this monitor, and should I need to replace the monitor that I amusing as a secondary monitor, I will likely purchase another one of this model! Again, Highly recommended and over all this monitor gets a full 5 Stars from me!
"""

In [31]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The reviewer purchased a Samsung curved screen monitor and was impressed with the picture quality and size. They had some issues with the packaging but overall highly recommend the monitor.


In [32]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The reviewer purchased a Samsung curved screen monitor with 4K support. The first monitor arrived with a broken screen, but Amazon promptly sent a replacement. Setup was easy, and the picture quality is stunning. The reviewer recommends the monitor for home or office use.


In [34]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value, \
Make it focus for the pricing department.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The reviewer was initially looking for a larger monitor with 4K support at a reasonable price. They were impressed by the curved screen and positive reviews. The monitor had a broken screen upon arrival, but the replacement was promptly sent. The setup was easy, but the base stand design was inconvenient. The picture quality was described as phenomenal, with vibrant colors and no distortion. The monitor lacks built-in speakers. Overall, the reviewer highly recommends the monitor for its excellent picture quality and reasonable price.


In [35]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The review mentions that the first monitor arrived with a broken screen, but Amazon provided a replacement the next day.


## ---------------------------------------------

# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

In [36]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{prod_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [37]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{prod_review}'''
"""
response = get_completion(prompt)
print(response)

positive


In [38]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{prod_review}'''
"""
response = get_completion(prompt)
print(response)

excited, impressed, curious, happy, recommended


In [39]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{prod_review}'''
"""
response = get_completion(prompt)
print(response)

No


LLMS

In [40]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{prod_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "monitor",
  "Brand": "Samsung"
}


In [42]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item
- Monitor Name

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item", "MonitorName" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{prod_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "monitor",
  "MonitorName": "unknown",
  "Brand": "Samsung"
}


In [43]:
story="""
CAIRO/JERUSALEM, Oct 27 (Reuters) - Projectiles hit two Egyptian Red Sea towns on Friday, sources and officials said, injuring six people and showing the risk of regional spillover from the Israel-Gaza conflict.
Israel's military blamed an "aerial threat" in the Red Sea region: a possible reference to Yemen's Iran-backed Houthi movement which is known to use drones.
Egyptian army spokesperson Colonel Gharib Abdel-Hafez said an "unidentified drone" crashed into a building adjacent to a hospital injuring the six in Taba, on the border with Israel, in the early hours.
Later, another projectile fell near an electricity plant in a desert area of the town of Nuweiba about 70 km (43 miles) from the border, two Egyptian security sources told Reuters, adding that they were still gathering more information.
There was no claim of responsibility.
Taba and Nuweiba, both in Egypt's Sinai Peninsula, are popular with tourists.
Witnesses in both places, who asked not to be named, confirmed hearing explosions and seeing smoke rising plus Egyptian warplanes flying overhead.
Without specifying the location, Israel's military spokesperson Daniel Hagari said combat helicopters were scrambled when "an aerial threat was spotted in the Red Sea region".
"To our understanding, the strike that took place in Egypt originated in this threat," he added in a televised briefing. "Israel will work with Egypt, and the United States, and bolster regional defences against threats from the Red Sea region."
The U.S. said last week a Navy warship in the Red Sea intercepted projectiles launched by the Houthis, potentially toward Israel.
Bordering both Gaza and Israel, Egypt is exposed to the conflict that blew up after Hamas' Oct. 7 assault on Israel and the subsequent bombardment of the Gaza Strip.
Cairo has been prominently advocating for aid flows into Gaza, the release of Hamas' hostages and a ceasefire.
Last weekend, several Egyptian border guards were injured after being accidentally hit by fragments of a shell from an Israeli tank. Israel apologised for the incident.
On Wednesday, Hamas said it had targeted the Israeli town of Eilat, adjacent to Taba across the border, with a missile in what appeared to be the Islamist group's longest-range Palestinian attack of the flare-up since Oct. 7.
Reporting by Emily Rose and Dan Williams in Jerusalem, Ahmed Mohamed Hassan, Nafisa Eltahir, Ahmed Tolba, Hatem Maher, and Mai Shams El-Din in Cairo, Yusri Mohamed in Ismailia, Nadine Awadalla and Ahmed Elimam in Dubai; writing by Rami Ayyub and Nafisa Eltahir; Editing by Michael Perry and Andrew Cawthorne.
"""

In [44]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.
Then format it in a topic_list = ["item1", "item2", ...] format.
Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

- Projectiles hitting Egyptian Red Sea towns
- Risk of regional spillover from Israel-Gaza conflict
- Possible involvement of Yemen's Iran-backed Houthi movement
- Injuries caused by an unidentified drone
- Israel's military response and cooperation with Egypt and the United States

topic_list = ["Projectiles", "Spillover risk", "Houthi movement", "Unidentified drone", "Military response"]


In [45]:
topic_list = ["Projectiles", "Spillover risk", "RozBlbn", "Batata", "pizza", "Houthi movement", "Unidentified drone", "Military response"]

In [46]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

[1, 1, 0, 0, 0, 1, 1, 0]


# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

In [19]:
prompt = f"""
Translate the following  text to Arabic and Egyptian Arabic.: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

Arabic: أريد أن أطلب كرة سلة
Egyptian Arabic: عايز أطلب كورة سلة


In [20]:
prompt = f"""
Translate the following from slang to a business letter: 
'Yo Khaled, whatsup, we have to finish the courier development before the end of the year, wake up the development team bro.'
"""
response = get_completion(prompt)
print(response)

Dear Khaled,

I hope this letter finds you well. I wanted to bring to your attention the urgency of completing the courier development project before the end of the year. It is crucial that we expedite the progress of the development team.

Thank you for your attention to this matter.

Sincerely,
[Your Name]


In [29]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I believe there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


Got this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  One </span><span style='color:green;font-weight:700;'>cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's </span><span style='color:green;font-weight:700;'>Additionally, it's </span>a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it though. </span><span style='color:green;font-weight:700;'>it. </span>I <span style='color:red;font-weight:700;text-decoration:line-through;'>think </span><span style='color:green;font-weight:700;'>believe </span>there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

## Customize the automated reply to a customer email

In [47]:
# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [48]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{review}'''
"""
response = get_completion(prompt)
sentiment = response
print(sentiment)

negative


In [49]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

We are sorry to hear about the price increase you noticed in December. We strive to provide competitive pricing for our products, and we understand your frustration. If you have any further concerns regarding pricing or any other issues, we encourage you to reach out to our customer service team. They will be more than happy to assist you.

We also appreciate your feedback regarding the base of the system. We continuously work to improve the quality of our products, and your comments will be taken into consideration for future enhancements.

We apologize for any inconvenience caused by the motor issue you encountered. Our customer service team is always available to assist with any warranty-related concerns. We understand that the warranty had expired, but we would still like to address this matter further. Please 

# Order Chatbot

In [3]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [1]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [6]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

C:\Users\Omar Hamad\AppData\Local\Temp\ipykernel_19716\3082588398.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


BokehModel(combine_events=True, render_bundle={'docs_json': {'a0995cf0-6162-4535-a3b4-66b6a5f4e6dc': {'version…

C:\Users\Omar Hamad\AppData\Local\Temp\ipykernel_19716\3082588398.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)